In [2]:
import pandas as pd
import numpy as np
import mysql.connector as sqlconnector

In [91]:
connection = connection = sqlconnector.connect(host='192.168.1.73',
                           database='presensi_epsindo',
                           user='root',
                           password='epsindo123')

In [65]:
def load_department(conn) :
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM department")
    result = cursor.fetchall()
    
    name = [result[i][1] for i in range(len(result))]
    ids = [result[i][0] for i in range(len(result))]

    department_df = pd.DataFrame({'id':ids, 'name':name})
    return department_df

load_department(connection)

,id,name
0,0,Warehouse
1,1,Other
2,2,Finance & Accounting
3,3,Sales Department
4,4,Service & Support Department
5,5,Marketing
6,6,Product Manager


In [20]:
df_participant
cursor = connection.cursor()

sql = "INSERT INTO participant (participant_id, name, department_id, email)  VALUES (%s, %s, %s, %s)"
values = []
for ind,val in df_participant.iterrows() :
    values.append((val['id_p'], val['Name'], val['dep_id'], val['email']))
print(values)
cursor.executemany(sql, values)
connection.commit()
print(cursor.rowcount, "rows executed.")

[(0, 'Chris Ariwibowo', 0, 'None'), (1, 'Mirsan Mirsan', 0, 'None'), (2, 'Slamat Riadi', 0, 'None'), (3, 'Sumarno Sumarno', 0, 'None'), (4, 'Bayu Bayu', 1, 'None'), (5, 'Yuni Aryani', 2, 'None'), (6, 'Jumyati Jumyati', 3, 'None'), (7, 'Ineke Sri Lestari', 2, 'None'), (8, 'Cahyanto Cahyanto', 4, 'None'), (9, 'Fachrudin Ali', 5, 'None'), (10, 'Nuryanti Nuryanti', 2, 'None'), (11, 'Setia Wayah Gutama', 4, 'None'), (12, 'harun boerhan', 4, 'None'), (13, 'Nurahdiatna Subagya', 4, 'None'), (14, 'Mark Kenny Williams', 3, 'None'), (15, 'Muh. Agung Fasrulhaq', 4, 'None'), (16, 'raisul amin', 4, 'None'), (17, 'Melissa Christina Tuwoh', 3, 'None'), (18, 'Lina Wirahadi', 6, 'None'), (19, 'Dwi Septia Ningsih', 0, 'None'), (20, 'Roby Roby', 1, 'None'), (21, 'Arrent Ong', 6, 'None')]


TypeError: 'int' object is not callable

In [21]:
print(cursor.rowcount, "rows executed.")

22 rows executed.


In [152]:
def load_participants(conn, department=-1):
    cursor = conn.cursor()

    if department < 0 :
        cursor.execute("SELECT * FROM participant")
    else :
        cursor.execute("SELECT * FROM participant WHERE department_id="+str(department))
    
    result = cursor.fetchall()
    
    name = [result[i][1] for i in range(len(result))]
    ids = [result[i][0] for i in range(len(result))]
    dep_ids = [result[i][2] for i in range(len(result))]
    emails = [result[i][3] for i in range(len(result))]

    participant_df = pd.DataFrame({'id':ids, 'name':name, 'dep_id':dep_ids, 'email':emails})
    return participant_df

load_participants(connection)

,id,name,dep_id,email
0,0,Chris Ariwibowo,0,None
1,1,Mirsan Mirsan,0,None
2,2,Slamat Riadi,0,None
3,3,Sumarno Sumarno,0,None
4,4,Bayu Bayu,1,None
5,5,Yuni Aryani,2,None
6,6,Jumyati Jumyati,3,None
7,7,Ineke Sri Lestari,2,None
8,8,Cahyanto Cahyanto,4,None
9,9,Fachrudin Ali,5,None


In [62]:
df_presensi = df_presensi.drop(columns=['ID'])
# df_presensi = df_presensi.fillna()
df_presensi['First Check In'] = pd.to_datetime(df['First Check In'], format='%I:%M %p').dt.strftime('%H:%M:%S')
df_presensi['Last Check Out'] = pd.to_datetime(df['Last Check Out'], format='%I:%M %p').dt.strftime('%H:%M:%S')
df_presensi = df_presensi.fillna(np.nan).replace([np.nan], [None])
df_presensi

,Date,Participant_id,Status,First Check In,Last Check Out
7,2022-07-26,0,IN,09:53:00,None
8,2022-07-26,1,IN,09:50:00,None
9,2022-07-26,2,IN,09:22:00,None
10,2022-07-26,3,IN,09:22:00,None
11,2022-07-26,4,IN,08:41:00,None
...,...,...,...,...,...
323,2022-07-01,11,Present,10:05:00,17:03:00
324,2022-07-01,18,Present,10:01:00,17:02:00
325,2022-07-01,13,Present,09:56:00,17:02:00
326,2022-07-01,15,Present,09:59:00,17:05:00


In [68]:
cursor = connection.cursor()

sql = "INSERT INTO presensi_table (date, participant_id, status, firstcheckin, lastcheckout)  VALUES (%s, %s, %s, %s, %s)"
values = []
for ind,val in df_presensi.iterrows() :
    # print(val.values)
    values = tuple(val.values)
    cursor.execute(sql, values)
connection.commit()
print(cursor.rowcount, "rows executed.")

1 rows executed.


In [103]:
from math import floor
from datetime import date


def format_timedelta(td):
    if pd.isnull(td) :
        return None
    else :
        minutes, seconds = divmod(td.seconds + td.days * 86400, 60)
        hours, minutes = divmod(minutes, 60)
        return '{:d}:{:02d}:{:02d}'.format(hours, minutes, seconds)

def total_late(fi) :
    if fi.total_seconds()/3600 >= 10:
        tolate = (fi.total_seconds() - (3600*10))/60
        return str(tolate) + " Minute"
    else :
        return 0

def load_presensi(conn, month='all', year=date.today().year):
    cursor = conn.cursor()
    monthdict = { 'January' : 1 ,
                  'February' : 2,
                  'March' : 3,
                  'April' : 4,
                  'May' : 5,
                  'June' : 6,
                  'July' : 7,
                  'August' : 8,
                  'September' : 9,
                  'October' : 10,
                  'November' : 11,
                  'December' : 12}
    if month=='all' :
        query = "SELECT * FROM presensi_table"
    else :
        query = "SELECT * FROM presensi_table WHERE MONTH(date)=" + str(monthdict[month]) +" AND YEAR(date)=" + str(year)
    cursor.execute(query)

    result = cursor.fetchall()
    df_presensi = pd.DataFrame(data=result, columns=['id','Date', 'Participant_id','status','firstcheckin', 'lastcheckout'])
    df_presensi['firstcheckinstr'] = df_presensi['firstcheckin'].apply(lambda x : format_timedelta(x))
    df_presensi['lastcheckoutstr'] = df_presensi['lastcheckout'].apply(lambda x : format_timedelta(x))
    df_presensi['total_late'] = df_presensi['firstcheckin'].apply(lambda x: total_late(x))
    return df_presensi

load_presensi(connection, month='January')

,id,Date,Participant_id,status,firstcheckin,lastcheckout,firstcheckinstr,lastcheckoutstr,total_late


In [99]:
from datetime import date
year=date.today().year
year

2022

In [90]:
df_leave

,ID,Name,Applied At,Leave,From,To,For,Session,Status,Emergency,Reason,Remark,Department,Branch,Attachment
0,1007,Melissa Christina Tuwoh,2022-07-14,ANNUAL,2022-07-29,2022-07-29,1.00 Days,Full Day,Pending Approval,NaN,Acara keluarga,NaN,Sales Department,NaN,NaN
1,4,Dwi Septia Ningsih,2022-07-19,ANNUAL,2022-07-25,2022-07-26,2.00 Days,Full Day,Approved,NaN,Ada urusan keluarga,NaN,Warehouse,NaN,NaN
2,1004,Cahyanto Cahyanto,2022-07-15,ANNUAL,2022-07-15,2022-07-15,1.00 Days,Full Day,Approved,Yes,Antar ibu ke poli Mata Katarak,NaN,Service & Support Department,NaN,√
3,1006,Nuryanti Nuryanti,2022-07-19,ANNUAL,2022-07-15,2022-07-15,1.00 Days,Full Day,Pending Approval,Yes,Sakit,NaN,Finance & Accounting,NaN,NaN
4,1002,Jumyati Jumyati,2022-07-14,ANNUAL,2022-07-15,2022-07-18,2.00 Days,Full Day,Approved,NaN,Ada keperluan keluarga,NaN,Sales Department,NaN,NaN
5,1,Chris Ariwibowo,2022-07-08,Bereavement,2022-07-11,2022-07-11,1.00 Days,Full Day,Approved,NaN,Keluarga,NaN,Warehouse,NaN,NaN
6,1013,Nurahdiatna Subagya,2022-06-25,ANNUAL,2022-07-08,2022-07-08,1.00 Days,Full Day,Cancelled,NaN,Pulang kampung idul adha,NaN,Service & Support Department,NaN,NaN
7,3,Slamat Riadi,2022-07-05,ANNUAL,2022-07-07,2022-07-07,1.00 Days,Full Day,Approved,NaN,Ada urusan keluarga,NaN,Warehouse,NaN,NaN
8,1014,Mark Kenny Williams,2022-06-23,ANNUAL,2022-06-27,2022-07-05,7.00 Days,Full Day,Approved,NaN,Family visit Canada,NaN,Sales Department,NaN,NaN


In [94]:
    df_leave_tb = df_leave_tb.drop(columns=['id_leave'])
    df_leave_tb

,participant_id,leave_kind,from,to,reason
0,17,ANNUAL,2022-07-29,2022-07-29,Acara keluarga
1,19,ANNUAL,2022-07-25,2022-07-26,Ada urusan keluarga
2,8,ANNUAL,2022-07-15,2022-07-15,Antar ibu ke poli Mata Katarak
3,10,ANNUAL,2022-07-15,2022-07-15,Sakit
4,6,ANNUAL,2022-07-15,2022-07-18,Ada keperluan keluarga
5,0,Bereavement,2022-07-11,2022-07-11,Keluarga
6,13,ANNUAL,2022-07-08,2022-07-08,Pulang kampung idul adha
7,2,ANNUAL,2022-07-07,2022-07-07,Ada urusan keluarga
8,14,ANNUAL,2022-06-27,2022-07-05,Family visit Canada


In [96]:
cursor = connection.cursor()

leave_values = []
for ind, val in df_leave_tb.iterrows():
    leave_values.append(tuple(val.values))
sql = "INSERT INTO leave_table (participant_id, leave_kind, leave_from, leave_to, reason) VALUES(%s, %s, %s, %s, %s)"
cursor.executemany(sql, leave_values)
connection.commit()

print(cursor.rowcount, " rows inserted.")


9  rows inserted.


In [104]:
df_presensi

,Date,Participant_id,Status,First Check In,Last Check Out
7,2022-07-26,0,IN,09:53:00,None
8,2022-07-26,1,IN,09:50:00,None
9,2022-07-26,2,IN,09:22:00,None
10,2022-07-26,3,IN,09:22:00,None
11,2022-07-26,4,IN,08:41:00,None
...,...,...,...,...,...
323,2022-07-01,11,Present,10:05:00,17:03:00
324,2022-07-01,18,Present,10:01:00,17:02:00
325,2022-07-01,13,Present,09:56:00,17:02:00
326,2022-07-01,15,Present,09:59:00,17:05:00


In [105]:
df_participant

,id_p,Name,dep_id,email
0,0,Chris Ariwibowo,0,None
1,1,Mirsan Mirsan,0,None
2,2,Slamat Riadi,0,None
3,3,Sumarno Sumarno,0,None
4,4,Bayu Bayu,1,None
5,5,Yuni Aryani,2,None
6,6,Jumyati Jumyati,3,None
7,7,Ineke Sri Lestari,2,None
8,8,Cahyanto Cahyanto,4,None
9,9,Fachrudin Ali,5,None


In [116]:
import datetime, calendar
y = 2022
m = 7
print(calendar.monthrange(y,m)[1])
days = [datetime.date(y, m, d+1) for d in range(calendar.monthrange(y,m)[1])]
print(days)

31
[datetime.date(2022, 7, 1), datetime.date(2022, 7, 2), datetime.date(2022, 7, 3), datetime.date(2022, 7, 4), datetime.date(2022, 7, 5), datetime.date(2022, 7, 6), datetime.date(2022, 7, 7), datetime.date(2022, 7, 8), datetime.date(2022, 7, 9), datetime.date(2022, 7, 10), datetime.date(2022, 7, 11), datetime.date(2022, 7, 12), datetime.date(2022, 7, 13), datetime.date(2022, 7, 14), datetime.date(2022, 7, 15), datetime.date(2022, 7, 16), datetime.date(2022, 7, 17), datetime.date(2022, 7, 18), datetime.date(2022, 7, 19), datetime.date(2022, 7, 20), datetime.date(2022, 7, 21), datetime.date(2022, 7, 22), datetime.date(2022, 7, 23), datetime.date(2022, 7, 24), datetime.date(2022, 7, 25), datetime.date(2022, 7, 26), datetime.date(2022, 7, 27), datetime.date(2022, 7, 28), datetime.date(2022, 7, 29), datetime.date(2022, 7, 30), datetime.date(2022, 7, 31)]


In [124]:
datetime.datetime.strptime(df_presensi.iloc[0]["Date"], "%Y-%m-%d").date() == datetime.date(2022, 7, 26)

True

In [125]:
df_presensi['datetimedate'] = df_presensi["Date"].apply(lambda x : datetime.datetime.strptime(x, "%Y-%m-%d").date())
df_presensi.head()

,Date,Participant_id,Status,First Check In,Last Check Out,datetimedate
7,2022-07-26,0,IN,09:53:00,None,2022-07-26
8,2022-07-26,1,IN,09:50:00,None,2022-07-26
9,2022-07-26,2,IN,09:22:00,None,2022-07-26
10,2022-07-26,3,IN,09:22:00,None,2022-07-26
11,2022-07-26,4,IN,08:41:00,None,2022-07-26


In [128]:
df_presensi["datetimedate"].values

array([datetime.date(2022, 7, 26), datetime.date(2022, 7, 26),
       datetime.date(2022, 7, 26), datetime.date(2022, 7, 26),
       datetime.date(2022, 7, 26), datetime.date(2022, 7, 26),
       datetime.date(2022, 7, 26), datetime.date(2022, 7, 26),
       datetime.date(2022, 7, 26), datetime.date(2022, 7, 26),
       datetime.date(2022, 7, 26), datetime.date(2022, 7, 26),
       datetime.date(2022, 7, 26), datetime.date(2022, 7, 26),
       datetime.date(2022, 7, 26), datetime.date(2022, 7, 26),
       datetime.date(2022, 7, 26), datetime.date(2022, 7, 25),
       datetime.date(2022, 7, 25), datetime.date(2022, 7, 25),
       datetime.date(2022, 7, 25), datetime.date(2022, 7, 25),
       datetime.date(2022, 7, 25), datetime.date(2022, 7, 25),
       datetime.date(2022, 7, 25), datetime.date(2022, 7, 25),
       datetime.date(2022, 7, 25), datetime.date(2022, 7, 25),
       datetime.date(2022, 7, 25), datetime.date(2022, 7, 25),
       datetime.date(2022, 7, 25), datetime.date(2022, 

In [148]:
import datetime, calendar
def load_monthly_table_data(df_participant, df_presensi, month=1, year=2022) :
    days = [datetime.date(y, m, d+1) for d in range(calendar.monthrange(y,m)[1])]
    weekdays = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
    dict_absen = {'Name':[]}

    dates = 0
    for day in days :
        wkday = weekdays[day.weekday()]
        dict_absen[day] = []
        dates += 1
    dict_absen['presensi_total'] = []
    dict_absen['absensi_total'] = []
    dict_absen['cuti_total'] = []

    # dict_absen
    for ind,val in df_participant.iterrows() :
        dict_absen['Name'].append(val['Name'])
        presentlist = df_presensi[df_presensi["Participant_id"] == val["id_p"]]
        daysindata = presentlist['datetimedate'].values
        presentotal = 0
        absentotal = 0
        for day in days:
            if day.weekday() == 5 or day.weekday() == 6 :
                dict_absen[day].append('L')

            elif day in daysindata :
                dict_absen[day].append('P')
                presentotal += 1

            else :
                dict_absen[day].append('A')
                absentotal += 1
        dict_absen['presensi_total'].append(presentotal)
        dict_absen['absensi_total'].append(absentotal)
        dict_absen['cuti_total'].append(0)

    return pd.DataFrame(dict_absen)

df_absen = load_monthly_table_data(df_participant, df_presensi, month=7, year=2022)
df_absen

,Name,2022-07-01,2022-07-02,2022-07-03,2022-07-04,2022-07-05,2022-07-06,2022-07-07,2022-07-08,2022-07-09,...,2022-07-25,2022-07-26,2022-07-27,2022-07-28,2022-07-29,2022-07-30,2022-07-31,presensi_total,absensi_total,cuti_total
0,Chris Ariwibowo,P,L,L,P,P,P,P,P,L,...,P,P,A,A,A,L,L,17,4,0
1,Mirsan Mirsan,P,L,L,P,P,P,P,P,L,...,P,P,A,A,A,L,L,18,3,0
2,Slamat Riadi,P,L,L,P,P,P,A,P,L,...,P,P,A,A,A,L,L,17,4,0
3,Sumarno Sumarno,P,L,L,P,P,P,P,P,L,...,P,P,A,A,A,L,L,18,3,0
4,Bayu Bayu,P,L,L,P,P,P,P,P,L,...,P,P,A,A,A,L,L,18,3,0
5,Yuni Aryani,P,L,L,P,P,P,P,P,L,...,P,P,A,A,A,L,L,18,3,0
6,Jumyati Jumyati,P,L,L,P,P,P,P,P,L,...,P,P,A,A,A,L,L,16,5,0
7,Ineke Sri Lestari,P,L,L,P,P,P,P,P,L,...,P,P,A,A,A,L,L,18,3,0
8,Cahyanto Cahyanto,P,L,L,P,P,P,P,P,L,...,P,P,A,A,A,L,L,14,7,0
9,Fachrudin Ali,P,L,L,A,A,A,A,P,L,...,P,P,A,A,A,L,L,13,8,0


In [151]:
weekdays = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
dict_rename = {}
day_col = 1
for x in df_absen.columns :
    if isinstance(x, datetime.date) :
        dict_rename[x] = weekdays[x.weekday()] +" "+ str(day_col)
        day_col += 1
df_absen.rename(columns = dict_rename, inplace=True)
df_absen


,Name,Fri 1,Sat 2,Sun 3,Mon 4,Tue 5,Wed 6,Thu 7,Fri 8,Sat 9,...,Mon 25,Tue 26,Wed 27,Thu 28,Fri 29,Sat 30,Sun 31,presensi_total,absensi_total,cuti_total
0,Chris Ariwibowo,P,L,L,P,P,P,P,P,L,...,P,P,A,A,A,L,L,17,4,0
1,Mirsan Mirsan,P,L,L,P,P,P,P,P,L,...,P,P,A,A,A,L,L,18,3,0
2,Slamat Riadi,P,L,L,P,P,P,A,P,L,...,P,P,A,A,A,L,L,17,4,0
3,Sumarno Sumarno,P,L,L,P,P,P,P,P,L,...,P,P,A,A,A,L,L,18,3,0
4,Bayu Bayu,P,L,L,P,P,P,P,P,L,...,P,P,A,A,A,L,L,18,3,0
5,Yuni Aryani,P,L,L,P,P,P,P,P,L,...,P,P,A,A,A,L,L,18,3,0
6,Jumyati Jumyati,P,L,L,P,P,P,P,P,L,...,P,P,A,A,A,L,L,16,5,0
7,Ineke Sri Lestari,P,L,L,P,P,P,P,P,L,...,P,P,A,A,A,L,L,18,3,0
8,Cahyanto Cahyanto,P,L,L,P,P,P,P,P,L,...,P,P,A,A,A,L,L,14,7,0
9,Fachrudin Ali,P,L,L,A,A,A,A,P,L,...,P,P,A,A,A,L,L,13,8,0


In [130]:
dict_abs = {'Name' : ['Yatna', 'Raisul'], 'Tue 1':['A', 'C']}
dacc = pd.DataFrame(dict_abs)
dacc

,Name,Tue 1
0,Yatna,A
1,Raisul,C


In [129]:
df_presensi

,Date,Participant_id,Status,First Check In,Last Check Out,datetimedate
7,2022-07-26,0,IN,09:53:00,None,2022-07-26
8,2022-07-26,1,IN,09:50:00,None,2022-07-26
9,2022-07-26,2,IN,09:22:00,None,2022-07-26
10,2022-07-26,3,IN,09:22:00,None,2022-07-26
11,2022-07-26,4,IN,08:41:00,None,2022-07-26
...,...,...,...,...,...,...
323,2022-07-01,11,Present,10:05:00,17:03:00,2022-07-01
324,2022-07-01,18,Present,10:01:00,17:02:00,2022-07-01
325,2022-07-01,13,Present,09:56:00,17:02:00,2022-07-01
326,2022-07-01,15,Present,09:59:00,17:05:00,2022-07-01


In [135]:
from datetime import datetime

days[3].weekday()

0